#  causality sentence Identifier

## 1.  load sentence

In [1]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

pm_DB = pd.read_csv('data/in/KNN_RAG/semeval_causalityDetection_cleaned.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]

instances = pm_DB['sentence'].to_list()

len(instances)

549

## 2. load fewshot database

In [2]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()


## 3. get causality label for sentence

In [3]:
from openai import OpenAI
client = OpenAI(api_key='enter your API key here..')


In [4]:
#Below are 10 example sentences which shows causality relation  with  their cause and effect 
          #phrase delimited between the XML tags <cause>, </cause> , <effect> and </effect>.
           #+ fewshot_examples
###model#gpt-3.5-turbo
def CS_Identifier(input_sentence, fewshot_examples):
    
    response = client.chat.completions.create(
      model="gpt-4o",
      response_format= {"type": "json_object"},
      messages=[
        {
          "role": "system",
          "content": '''
          You will be given a sentence, and your task is to identify whether the sentence shows a 
          causality relation or not. If the sentence shows a causality relation, you can output 1;
          if it doesn't, output 0. 
          Your output should be in json format: {output :  }.
          
          Hint: If the sentence shows causality relation, you will be able to identify a pair of 
          phrase in the sentence where one phrase will be the cause of the other phrase(effect).
          
          Below are 10 example sentences which shows causality relation  with  their cause and effect 
          phrase delimited between the XML tags <cause>, </cause> , <effect> and </effect>.
           ''' + fewshot_examples
          
            
        },
        {
          "role": "user",
          "content": input_sentence 
        }
      ],
      temperature=0.0,
      seed = 1,
      max_tokens=100,
      top_p=1
    )
    
    return response.choices[0].message.content


In [5]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
fewshot_examples_list=[]
causality_identifier=[]
fewshotfound_inputsentence_index=[]
for i in range (len(pm_DB)):
    input_sentence = pm_DB['sentence'][i]
    # few shot examples augmentation.
    
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_examples = [examples[j] for j in most_similar_indices]

    fewshot_examples = {f'example {k + 1}': sentence for k, sentence in enumerate(fewshot_examples)}
    fewshot_examples = json.dumps(fewshot_examples)   
    fewshot_examples_list.append(fewshot_examples)

    #####################
    #print('input_sentence : ', input_sentence)

    print('fewshot_examples : ', fewshot_examples)
    
    # output generation
    out = CS_Identifier(input_sentence, fewshot_examples)
    json_out = json.loads(out)
    causality_identifier_out = json_out['output']
    causality_identifier.append(int(causality_identifier_out))
    
    
    print(i)
    print(causality_identifier_out)
    
    

fewshot_examples :  {"example 1": "The pet was overly sensitive, and thus <cause>the ticks</cause> and fleas caused <effect>rashes</effect> and discomfort.", "example 2": "\"Canine <cause>flea</cause> <effect>infestation</effect> is caused by fleas, which are small, wingless blood-sucking insects.\"", "example 3": "\"Food <cause>poisoning</cause> is perhaps the commonest cause of <effect>abdominal pain</effect>, especially in the young dog or the scavenger type.\"", "example 4": "\"The <effect>pain</effect> was caused by the <cause>parasite</cause> entering the body through the skin during bathing or drinking of infested water.\"", "example 5": "\"<effect>Convulsions</effect> that occur after DTaP are caused by a <cause>fever</cause>.\"", "example 6": "\"It seems like the fever <effect>heat</effect> got pushed out from the <cause>body</cause> as rashes.\"", "example 7": "<cause>Cells damaged or killed</cause> by flu result in <effect>the production</effect> of interferon which is one o

5
0
fewshot_examples :  {"example 1": "\"And it is why we are even prouder to announce that the appeal - thanks to the generosity of our readers - has raised 26,000 pounds, despite the <effect>hardship</effect> that has been caused by the <cause>recession</cause>.\"", "example 2": "To that end, <cause>the coalition</cause> has established <effect>some local recreational activities</effect>, including Laser Tag and paintball.", "example 3": "\"All the <effect>profits</effect> from the <cause>books</cause> listed below are donated to the Norman Historic Preservation Program, Inc. (NHPP) to be used toward the renovation of the old Norman High School as a children's educational center and museum.\"", "example 4": "\"An organization with the mission of allowing a safe haven for parents to share their <effect>grief</effect> after the <cause>death</cause> of a child.\"", "example 5": "\"The <effect>shooting</effect> stemmed from an <cause>argument</cause> the two students had \"in the communi

10
1
fewshot_examples :  {"example 1": "\"That year, summer students at the VLA made the first discovery of radio <effect>emission</effect> from a brown <cause>dwarf</cause>.\"", "example 2": "\"The <cause>movement</cause> developed from the <effect>rediscovery</effect> by European scholars of many Greek and Roman texts.\"", "example 3": "That year, summer students at the VLA made the first discovery of <effect>radio emission</effect> from <cause>a brown dwarf</cause>.", "example 4": "\"The <effect>light</effect> that is currently reaching the Earth was generated in the <cause>sun</cause> approximately 100000 years ago.\"", "example 5": "\"The <effect>movement</effect> of the flag on the surface of the moon was caused by the <cause>vaccum</cause>.\"", "example 6": "\"Six glowing disks await <effect>activation</effect> by the <cause>footsteps</cause> of desert wanderers.\"", "example 7": "\"This photograph shows the <effect>vortices</effect> created by the water strider's <cause>legs</c

16
1
fewshot_examples :  {"example 1": "\"However, the planning <cause>proposal</cause> has generated <effect>opposition</effect> from locals who have voiced fears over health issues.\"", "example 2": "\"The <effect>accolade</effect> was decided upon after an intense <cause>discussion</cause> between about 200 members.\"", "example 3": "However, <cause>the planning proposal</cause> has generated <effect>opposition</effect> from locals who have voiced fears over health issues..\n\t\t", "example 4": "\"It was also felt necessary at this time to draw up a similar document, dealing not with the prevention of marine <effect>pollution</effect> by <cause>dumping</cause>, but instead with the prevention of marine pollution by discharges of dangerous substances from land-based sources, watercourses or pipelines.\"", "example 5": "It was also felt necessary at this time to draw up a similar document, dealing not with the prevention of <effect>marine pollution</effect> by <cause>dumping</cause>, 

21
0
fewshot_examples :  {"example 1": "\"In addition, a brief <cause>biography</cause> establishing her <effect>credentials</effect> was included at the end of the report.\"", "example 2": "In addition, <cause>a brief biography</cause> establishing <effect>her credentials</effect> was included at the end of the report.", "example 3": "\"The <cause>presentation</cause> generated <effect>debate</effect> and there was general discussion on how journalists can better equip themselves to handle the challenges.\"", "example 4": "\"The <effect>accolade</effect> was decided upon after an intense <cause>discussion</cause> between about 200 members.\"", "example 5": "\"When deliberations began, the <cause>crowd</cause> of people jamming the hallways radiated <effect>optimism</effect>.\"", "example 6": "When deliberations began, <cause>the crowd</cause> of people jamming the hallways radiated <effect>optimism</effect>.", "example 7": "\"The <effect>process</effect> departs from an <cause>inspira

...

523
0
fewshot_examples :  {"example 1": "\"In one case, the <cause>waterjet</cause> caused a large <effect>gouge</effect> on the bottom side of a a copper/ wood/copper piece.\"", "example 2": "\"The metal <cause>ball</cause> makes a ding ding ding <effect>noise</effect> when it swings back and hits the metal body of the lamp.\"", "example 3": "\"A <cause>lamp</cause> emiting a strong white <effect>light</effect> is placed on an mobile block makes it possible to the scanner to sweep all the document.\"", "example 4": "\"The design of the shape of the can ends moreover has a marked effect on their ability to resist <effect>deformation</effect> from internal <cause>pressure</cause>.\"", "example 5": "The design of the shape of the can ends moreover has a marked effect on their ability to resist <effect>deformation</effect> from <cause>internal pressure</cause>.", "example 6": "\"To determine, once and for all, how the <effect>manakin</effect> was making its bizarre <cause>sounds</cause>, 

528
0
fewshot_examples :  {"example 1": "\"The rescue teams found 17 of the 25 crew thrown into the sea when the tanker caught <effect>fire</effect> after the <cause>collision</cause>.\"", "example 2": "\"The <cause>sinking</cause> resulted in the <effect>loss</effect> of 112 lives, including those of 28 Americans.\"", "example 3": "\"The <cause>weather</cause> resulted in several deaths and hundreds of <effect>accidents</effect>.\"", "example 4": "\"The <cause>debris</cause> caused a <effect>crash</effect> on the Schuylkill, and it has been closed for hours because of the mishap.\"", "example 5": "\"The <effect>fire</effect> started with an <cause>explosion</cause> in the cargo hold and spread across the ship within 10 minutes.\"", "example 6": "\"A <effect>fire</effect> triggered by the <cause>blasts</cause> damaged eight buildings at the plant, including one that was burned down.\"", "example 7": "\"The continuous <cause>rains</cause> had caused the <effect>overflow</effect> of Sula

533
1
fewshot_examples :  {"example 1": "\"The <cause>earthquake</cause> caused a massive <effect>avalanche</effect> on the northern slope of Mount Huascar.\"", "example 2": "\"The <cause>debris</cause> caused a <effect>crash</effect> on the Schuylkill, and it has been closed for hours because of the mishap.\"", "example 3": "\"The <cause>snowstorm</cause> caused a major <effect>catastrophe</effect> on the West Virginia Turnpike.\"", "example 4": "\"The <cause>weather</cause> resulted in several deaths and hundreds of <effect>accidents</effect>.\"", "example 5": "\"The <cause>accident</cause> caused a major traffic <effect>snarl</effect> on the arterial road.\"", "example 6": "That coupled with <effect>the death</effect> and destruction caused by <cause>the storm</cause> was a very traumatic experience for these residents.", "example 7": "\"A <effect>fire</effect> triggered by the <cause>blasts</cause> damaged eight buildings at the plant, including one that was burned down.\"", "examp

539
0
fewshot_examples :  {"example 1": "\"In a poetic twist, the <effect>tsunami</effect> triggered by the rock <cause>band</cause> forced many American jazz and blues singers to seek work in the United Kingdom, which is where Anderson found a receptive audience in 1965.\"", "example 2": "Anthony finds inspiration in a beautiful, enigmatic older woman who teaches him to follow his heart and find <effect>happiness</effect> through <cause>swing dancing</cause>.", "example 3": "\"The <effect>concerto</effect> comes from the <cause>pop</cause> of gas bubbles escaping the joints, snapping tendons and ligaments, and rickety arthritic joints.\"", "example 4": "\"The <effect>singing</effect> comes from the substantial <cause>figure</cause> of Mole - Paul Smith in the normal world - who crams into his days a full-time job.\"", "example 5": "\"Anthony finds inspiration in a beautiful, enigmatic older woman who teaches him to follow his heart and find <effect>happiness</effect> through swing <ca

544
1
fewshot_examples :  {"example 1": "\"A magnetic bracelet helps the <effect>pain</effect> from <cause>stroke</cause> in your arm.\"", "example 2": "A magnetic bracelet helps <effect>the pain</effect> from <cause>stroke</cause> in your arm.", "example 3": "\"Many <effect>shoulder problems</effect> are caused by the <cause>breakdown</cause> of soft tissues in the shoulder region.\"", "example 4": "A 60 year-old woman with chronic renal failure developed acute <effect>proximal muscle weakness</effect> after receiving a regular dosage of <cause>colchicine</cause>.", "example 5": "Shortly after chemotherapy and an injection of <cause>pegfilgrastim</cause>, the patient developed poorly defined, rapidly progressive erythema, <effect>edema</effect>, and pain in his right forearm.", "example 6": "A 79-year-old man with ischemic heart disease, chronic atrial fibrillation, chronic renal failure, hypothyroidism, and gout arthritis was hospitalized because of <effect>fatigue</effect>, myalgia,

In [6]:
1524

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


1524

In [7]:
len(causality_identifier)

549

In [8]:
len(fewshotfound_inputsentence_index)

0

In [9]:
#get  pattern_matched label
pattern_matched = [0]*(len(pm_DB))

# Update values to 1 at specified indexes
for index in fewshotfound_inputsentence_index:
    if index < len(pattern_matched):
        pattern_matched[index] = 1
pattern_matched

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [10]:
pm_DB['GPT_causality_label'] = causality_identifier 
pm_DB['fewshot_examples'] = fewshot_examples_list 

pm_DB['patternmatcher_label'] = pattern_matched
pm_DB

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings,GPT_causality_label,fewshot_examples,patternmatcher_label
0,0,0,0,Dogs develop a fever from stress and/or pain s...,1,"[-0.0036133944522589445, 0.0012565960641950369...",1,"{""example 1"": ""The pet was overly sensitive, a...",0
1,1,1,1,The new pencil and graphite drawing is the pai...,0,"[-0.02040475234389305, 0.022646330296993256, 0...",0,"{""example 1"": ""After several years of self-obs...",0
2,2,2,2,The deleveraging triggered by the crisis impli...,1,"[-0.02002304047346115, -0.031225474551320076, ...",1,"{""example 1"": ""\""The <effect>benefits</effect>...",0
3,3,3,3,The fat and cholesterol cause heart disease; t...,1,"[0.002320877043530345, -0.014957467094063759, ...",1,"{""example 1"": ""\""<cause>Fats</cause> from meat...",0
4,4,4,4,"The researchers placed the compound in a tube,...",0,"[-0.019478240981698036, 0.0026719982270151377,...",0,"{""example 1"": ""\""The <effect>magnetic field</e...",0
...,...,...,...,...,...,...,...,...,...
544,544,621,621,Neuralgia (TN) is the most common cause of chr...,1,"[-0.010079591535031796, 0.014008276164531708, ...",1,"{""example 1"": ""<cause>Neuralgia</cause> is th...",0
545,545,622,622,Muscle fatigue is the number one cause of arm ...,1,"[-0.011783338151872158, -0.007658560294657946,...",1,"{""example 1"": ""\""A magnetic bracelet helps the...",0
546,546,623,623,The movie featured a mutated sheep as the titu...,0,"[-0.015397767536342144, -0.01711292937397957, ...",0,"{""example 1"": ""\""The film also features docume...",0
547,547,624,624,A plasma was generated by an electric discharg...,1,"[-0.011884856037795544, 0.02052135579288006, 0...",1,"{""example 1"": ""\""The primary electron <effect>...",0


In [11]:
# Function to set 'prediction' column based on conditions
def set_prediction(row):
    if row['GPT_causality_label'] == 1 and row['patternmatcher_label'] == 1:
        return 1
    else:
        return 0

# Apply the function to each row and create the 'prediction' column
pm_DB['prediction'] = pm_DB.apply(set_prediction, axis=1)
pm_DB

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings,GPT_causality_label,fewshot_examples,patternmatcher_label,prediction
0,0,0,0,Dogs develop a fever from stress and/or pain s...,1,"[-0.0036133944522589445, 0.0012565960641950369...",1,"{""example 1"": ""The pet was overly sensitive, a...",0,0
1,1,1,1,The new pencil and graphite drawing is the pai...,0,"[-0.02040475234389305, 0.022646330296993256, 0...",0,"{""example 1"": ""After several years of self-obs...",0,0
2,2,2,2,The deleveraging triggered by the crisis impli...,1,"[-0.02002304047346115, -0.031225474551320076, ...",1,"{""example 1"": ""\""The <effect>benefits</effect>...",0,0
3,3,3,3,The fat and cholesterol cause heart disease; t...,1,"[0.002320877043530345, -0.014957467094063759, ...",1,"{""example 1"": ""\""<cause>Fats</cause> from meat...",0,0
4,4,4,4,"The researchers placed the compound in a tube,...",0,"[-0.019478240981698036, 0.0026719982270151377,...",0,"{""example 1"": ""\""The <effect>magnetic field</e...",0,0
...,...,...,...,...,...,...,...,...,...,...
544,544,621,621,Neuralgia (TN) is the most common cause of chr...,1,"[-0.010079591535031796, 0.014008276164531708, ...",1,"{""example 1"": ""<cause>Neuralgia</cause> is th...",0,0
545,545,622,622,Muscle fatigue is the number one cause of arm ...,1,"[-0.011783338151872158, -0.007658560294657946,...",1,"{""example 1"": ""\""A magnetic bracelet helps the...",0,0
546,546,623,623,The movie featured a mutated sheep as the titu...,0,"[-0.015397767536342144, -0.01711292937397957, ...",0,"{""example 1"": ""\""The film also features docume...",0,0
547,547,624,624,A plasma was generated by an electric discharg...,1,"[-0.011884856037795544, 0.02052135579288006, 0...",1,"{""example 1"": ""\""The primary electron <effect>...",0,0


In [12]:
#get accuracy for fewshot_semeval_causalityDetection: b/w label and GPT_causality_label 

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score

# Calculate precision, recall, and F1 score
precision = precision_score(pm_DB['label'], pm_DB['GPT_causality_label'])
recall = recall_score(pm_DB['label'], pm_DB['GPT_causality_label'])
f1 = f1_score(pm_DB['label'], pm_DB['GPT_causality_label'])
accuracy = accuracy_score(pm_DB['label'], pm_DB['GPT_causality_label'])

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.8925318761384335
F1 Score: 0.8757894736842106
Precision: 0.9203539823008849
Recall: 0.8353413654618473


In [ ]:

#filtered_rows = pm_DB[(pm_DB['GPT_causality_label'] == 1) & (pm_DB['patternmatcher_label'] == 1)]
#filtered_rows.to_csv('data/out/alzeimers_causalityrelation.csv')

#pm_DB.to_csv('data/out/RAG_causality/semeval/fewshotcomb_semeval_causalityDetection.csv')

In [ ]:
pm_DB.to_csv('data/out/RAG_causality/task1/GPT4/KNN_ADE_causalityDetection.csv')#RAG, rand,zero, ADE, SCITE, sem

In [ ]:
#pm_DB.to_csv('data/out/RAG_causality/task1/errorAnalysis/KNN_sem_causalityDetection.csv')#RAG, rand,zero, ADE, SCITE, sem